# Use machine learning to evaluate the psychosis dataset

In [2]:
from __future__ import division

from sklearn.decomposition import FactorAnalysis, PCA
from sklearn.model_selection import cross_val_score
from misc.prog import log_progress as lp
import statsmodels.formula.api as smf
from numpy import transpose as T
import matplotlib.pyplot as plt
from collections import Counter
import statsmodels.api as sm
import scipy.stats as stats
from misc import neuropsych
import matplotlib as mpl
import sklearn.cluster
import seaborn as sns
import datetime as dt
import pandas as pd
import numpy as np
import palettable
import warnings
import scipy
import time
import sys
import os

%matplotlib inline
%load_ext rpy2.ipython

#cols = palettable.colorbrewer.qualitative.Dark2_8.hex_colors
cols = palettable.tableau.ColorBlind_10.hex_colors
cols += palettable.tableau.PurpleGray_6.hex_colors
cols += palettable.tableau.Tableau_10.hex_colors

#warnings.filterwarnings("ignore")
CONDIR = os.environ.get("CONDIR")

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


## Read in training dataset

Read in connectomes

In [3]:
results = pd.read_csv(os.path.join(CONDIR,'derivatives/connectome_results.csv'))
results = results.drop('Unnamed: 0',axis=1)

 From now on, we only work with the training dataset

In [4]:
CONDIR = '/scratch/users/jdurnez/Psychosis_derivatives/connectivity_Joke/'
RESTABLE = pd.read_csv(os.environ.get("TRAINTABLE"))
#RESTABLE = RESTABLE[RESTABLE.is_this_subject_a_patient==888]

## Prepare neuropsych measures

First, I'm making a table with the codes, the labels and the scale of all neuropsych variables.

In [5]:
reload(neuropsych)

def get_tables():
# read in redcap
    RC = pd.read_csv(os.environ.get("REDCAPTABLE"),low_memory=False)
    RCinstr = neuropsych.redcap_instruments(RC)

    # read in labels of RC
    text_file = open(os.path.join(os.environ.get("CODEDIR"),"misc/neuropsych_labels.txt"),'r')
    lines = text_file.read().split("\n")

    labeltable = pd.DataFrame([RC.columns,lines])
    labeltable = labeltable.transpose()
    labeltable.columns = ['code','label']
    labeltable['scale'] = [k for x in labeltable.code for k,v in RCinstr.iteritems() if x in v]
    return RC, labeltable

RC, labeltable = get_tables()

labeltable[:5]


    ...getting redcap instruments...


,code,label,scale
0,subject_id,Subject,subject_status
1,redcap_data_access_group,Data Access Group,subject_status
2,is_this_subject_a_patient,"Is this subject a patient, control or drop?",subject_status
3,date_testing,Date of Ratings Testing,subject_status
4,date_nc_testing,Date of Neurocognitive Testing,subject_status


Now, I'm subsetting all psych variables, to include only certain scales, an only numerical values.  Then, I'm standardising the table, so that the weights of any data reduction are comparable.

In [ ]:
# get a subset of all neuropsych measures 

def subset_tables(include=None,RC=None, labeltable=None):
    if RC==None or labeltable==None:
        RC, labeltable = get_tables()     

    if include == None:
        include = [
            'legal_issues',
            'wasi',
            'bprse',
            'hamd',
            'ymrs',
            'ldps',
            'family_history_assessment',
            'ctq',
            'thq',
            'scid',
            'scid_face'
            ]

    # function get_measures_subset gets all non-text questions of the questionnaire
    # (without 'has this instr been collected' and 'iscomplete')
    neuropsych_cols = neuropsych.get_measures_subset(include=include)
    lbl_id = np.sort([np.where(x==labeltable.code)[0][0] for x in neuropsych_cols])

    #subset labeltable
    labeltable = labeltable.iloc[lbl_id]
    print("There are %i variables considered."%len(labeltable))

    #subset subjectstable and standardise
    FA_table = RESTABLE[labeltable.code]
    FA_table = (FA_table - FA_table.mean()) / (FA_table.max() - FA_table.min())
    FA_table = FA_table.fillna(0)
    
    return labeltable, FA_table

labeltable, FA_table = subset_tables()
FA_table[:5]



In [ ]:
len(labeltable)

## Data cleaning

Some columns pose problems for factor analysis (matrix singularity):
1. There seem to be columns that are empty for all subjects.  These should be removed.
2. There is an extremely high collinearity for certain columns (often _severity_ and _duration_ of same variable).  You can see these in the correlation plot below: the little brown squares in the ldps scale.  These should be removed or be summarised.

In [ ]:
cormat = FA_table.corr()
cormat.fillna(0)
unique_labels = np.unique(labeltable.scale)
labels_num = [np.where(unique_labels==x)[0].tolist()[0] for val,x in enumerate(labeltable.scale)]
major_ticks = [np.min(np.where(np.array(labels_num)==x))-1 for x in range(len(np.unique(labeltable.scale)))]
minor_ticks = [np.mean(np.where(np.array(labels_num)==x))-1 for x in range(len(np.unique(labeltable.scale)))]
fig = plt.figure(figsize=(8, 6), dpi= 100, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111)
ax1 = ax.imshow(cormat,cmap = "PuOr_r",vmin=-0.2,vmax=1,aspect='auto',interpolation='nearest')
ax.set_title('correlations between neuropsych measures')
ax.set_xticks(major_ticks,minor=False)
ax.set_xticks(minor_ticks,minor=True)
ax.set_xticklabels(unique_labels,minor=True,rotation=90)
ax.set_xticklabels(unique_labels,minor=False,visible=False)
ax.set_yticks(major_ticks,minor=False)
ax.set_yticks(minor_ticks,minor=True)
ax.set_yticklabels(unique_labels,minor=True)
ax.set_yticklabels(unique_labels,minor=False,visible=False)
plt.colorbar(ax1)
plt.tight_layout()

In [ ]:
def clean_tables(labeltable, FA_table):
    # append Nan columns
    dropvars = []
    for column in FA_table:
        FAnan = np.sum(FA_table[column]==0)
        if FAnan==len(FA_table):
            dropvars.append(column)

    # drop variables with too high correlations
    cormat = FA_table.corr()
    idx = np.where(cormat > 0.95)
    collin = [[x,y] for x,y in zip(idx[0],idx[1]) if x!=y and np.sort([x,y]).tolist()==[x,y]]
    for pair in collin:
        x = pair[0]
        y = pair[1]
        print("Correlation between %s and %s: %f"%(list(FA_table)[x],list(FA_table)[y],np.array(cormat)[x,y]))
        vardrop = list(FA_table)[y]
        dropvars.append(vardrop)
    
    labeltable = labeltable.reset_index()
    delvar = [np.where(x==labeltable.code)[0][0] for x in dropvars]
    labeltable.drop(delvar,inplace=True)

    FA_table = FA_table[labeltable.code]
    FA_table = (FA_table - FA_table.mean()) / (FA_table.max() - FA_table.min())
    FA_table = FA_table.fillna(0)

    return labeltable, FA_table

In [ ]:
include = [
#    'legal_issues',
#     'wasi',
#     'bprse',
#     'hamd',
#     'ymrs',
#     'ldps',
     'family_history_assessment',
#     'ctq',
#     'thq',
#     'scid',
#     'scid_face'
    ]
labeltable, legal_table = subset_tables(include=include)
labeltable, legal_table = clean_tables(labeltable, legal_table)


## look at data reduction of legal_issues

In [ ]:
%%R -i legal_table -o factortable

maxfac <- min(dim(legal_table)[2],50)

library(psych)
factortable <- nfactors(legal_table,n=maxfac,rotation='varimax')$vss.stats
factortable['eigen'] <- eigen(cor(legal_table))$values[1:maxfac]

parallel <- fa.parallel(legal_table,plot=FALSE)
factortable['parallel_data'] <- parallel$fa.values[1:maxfac]
factortable['parallel_sim'] <- parallel$fa.sim[1:maxfac]

vss <- vss(legal_table,n=maxfac,rotate='varimax',plot=FALSE)
factortable['map'] <- vss$map


In [ ]:
def evaluate_factors(factortable):
    # chi square of residual matrix
    pvals = [stats.chi2.pdf(x,y) for x,y in zip(factortable.chisq,factortable.dof)]
    num_c = sum(np.array(pvals)<0.05)
    print("Based on the likelihood ration test, Model(N)-Model(0), the minimum number of factors is %i"%num_c)

    # likelihood ration
    difs = np.diff(factortable.chisq)
    difdofs = np.diff(factortable.dof)
    pvals = [stats.chi2.pdf(-x,-y) for x,y in zip(difs,difdofs)]
    num_c = sum(np.array(pvals)<0.05)
    print("Based on the likelihood ration test, Model(b)-Model(n-1), the minimum number of factors is %i"%(num_c+1))

    # RMSEA
    num_r = sum(factortable.RMSEA>0.05)
    print("Based on the RMSEA, the minimum number of factors is %i"%(num_r))


    #kaiser criterion: number of eigenvalues > 1
    num_k = sum(factortable['eigen']>1)
    print("Based on the kaiser criterion, the minimum number of factors is %i"%(num_r))

    # BIC
    num_k = np.where(np.min(factortable.BIC) == factortable.BIC)[0][0]+1
    print("Based on the BIC, the minimum number of factors is %i"%(num_k))
    num_k = np.where(np.min(factortable.eBIC) == factortable.eBIC)[0][0]+1
    print("Based on the empirical BIC, the minimum number of factors is %i"%(num_k))

    # complexity
    num_c = np.where(np.max(factortable.complex)==factortable.complex)[0][0]+1
    print("Based on the complexity, the minimum number of factors is %i"%(num_c))

    # SRMR (standardised root mean square residual)
    num_c = sum(factortable.SRMR<0.08)
    print("Based on the standardised root mean square residual, the minimum number of factors is %i"%(num_c))

    # parallel analysis: extract factors until eigen values of data < eigen vanlues of random data
    num_c = np.min(np.where(factortable.parallel_data<factortable.parallel_sim)[0])+1
    print("Based on parallel analysis, the minimum number of factors is %i"%(num_c))

    # map: minimum average partial correlation
    num_c = np.where(np.min(factortable.map)==factortable.map)[0][0]
    print("Based on the minimum average partial correlation method, the minimum number of factors is %i"%(num_c))

In [ ]:
table = legal_table

In [ ]:
evaluate_factors(factortable)

In [ ]:
n_components = np.arange(0,14,1)
fa = FactorAnalysis()
fa.svd_method='randomized'
fa.iterated_power=3
fa.random_state = 100

fa_scores = []
maxlog = []
for n in lp(n_components):
    fa.n_components = n
    fa_scores.append(np.mean(cross_val_score(fa, legal_table)))
    fa.fit(X)
    maxlog.append(fa.loglike_[-1])

In [ ]:
fa_z = (np.array(fa_scores)-np.mean(fa_scores))/np.std(fa_scores)
maxlog_z = (np.array(maxlog)-np.mean(maxlog))/np.std(maxlog)
n_components_fa = n_components[np.argmax(fa_scores)]
n_components_ML = n_components[np.argmax(maxlog_z)]

plt.figure()
plt.plot(n_components, maxlog_z, cols[0], label='likelihood')
plt.axvline(n_components_ML, color=cols[0],
            label='FactorAnalysis loglikelihood: %d' % n_components_fa,
            linestyle='--')
plt.plot(n_components, fa_z, cols[1], label='FA CV scores')
plt.axvline(n_components_fa, color=cols[1],
            label='FactorAnalysis CV: %d' % n_components_fa,
            linestyle='--')

plt.xlabel('nb of components')
plt.ylabel('Standardized scores')
plt.legend(loc='lower right')
plt.title("Number of components based on cross-validation and maximum likelihood")

All metrics point in different directions, but I will follow the BIC and the cross-validation that indicate that 2 factors should do.

In [ ]:
%%R -i legal_table -o fa_loadings

library(psych)

factortable <- factanal(legal_table,factors=2,rotation='varimax')
fa_loadings = factortable$loadings

The factor analysis seems to indicate that the first factor indicates the presence of serious legal problems, while the second rather points to small legal problems.

In [ ]:
minor_ticks = np.arange(len(labeltable.code))

labelnew = [x[:50] for x in labeltable.label]
fig = plt.figure(figsize=(5, 6), dpi= 100, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111)
ax1 = ax.imshow(fa_loadings,
                aspect='auto',interpolation='nearest',cmap='coolwarm')
ax.set_yticks(np.arange(fa_loadings.shape[0]),minor=True)
ax.set_yticklabels(labelnew,minor=True)
plt.colorbar(ax1)


coldict = {k:cols[idx] for idx,k in enumerate(np.unique(labeltable.scale))}
labeltable['colors'] = [coldict[x] for x in labeltable.scale]
for ytick, color in zip(ax.get_yticklabels(minor=True), labeltable.colors):
    ytick.set_color(color)

plt.tight_layout()

In [ ]:
fa_loadings.shape

In [ ]:
labeltable = labeltable.reset_index()
delvar = [np.where(x==labeltable.code)[0][0] for x in dropvars]
labeltable.drop(delvar,inplace=True)

FA_table = FA_table[labeltable.code]
FA_table = (FA_table - FA_table.mean()) / (FA_table.max() - FA_table.min())
FA_table = FA_table.fillna(0)

print("These variables are dropped: \n%s"%"\n".join(dropvars))

In [ ]:
print("There are %i variables left in the neuropsych measurements"%len(labeltable))

In [ ]:
cormat = FA_table.corr()
cormat.fillna(0)
unique_labels = np.unique(labeltable.scale)
labels_num = [np.where(unique_labels==x)[0].tolist()[0] for val,x in enumerate(labeltable.scale)]
major_ticks = [np.min(np.where(np.array(labels_num)==x))-1 for x in range(len(np.unique(labeltable.scale)))]
minor_ticks = [np.mean(np.where(np.array(labels_num)==x))-1 for x in range(len(np.unique(labeltable.scale)))]
fig = plt.figure(figsize=(6, 5), dpi= 100, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111)
ax1 = ax.imshow(cormat,cmap = "PuOr_r",vmin=-0.2,vmax=1,aspect='auto',interpolation='nearest')
ax.set_title('correlations between neuropsych measures')
ax.set_xticks(major_ticks,minor=False)
ax.set_xticks(minor_ticks,minor=True)
ax.set_xticklabels(unique_labels,minor=True,rotation=90)
ax.set_xticklabels(unique_labels,minor=False,visible=False)
ax.set_yticks(major_ticks,minor=False)
ax.set_yticks(minor_ticks,minor=True)
ax.set_yticklabels(unique_labels,minor=True)
ax.set_yticklabels(unique_labels,minor=False,visible=False)
plt.colorbar(ax1)
plt.tight_layout()

In [ ]:
FA_table.to_csv("FA_table.csv")

## Baseline prediction accuracy based on neuropsych variables

Right now, the SVM is having extremely high classification accuracy.  Not really that much of a surprise, given that some forms were only filled out by psych patients (verify !) and the presence of hallucinations is pretty obvious :) .

In [ ]:
y = psy_labels
X = np.array(FA_table)

from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

cv = ShuffleSplit(n_splits=50, test_size=0.2, random_state=100)
sns.set_style("white")

def plot_learning_curve(estimator,title,X,y,ylim,cv=None,n_jobs=3,train_sizes = np.linspace(0.1,1.0,10)):
    plt.figure()
    plt.title(title)
    plt.ylim(ylim)
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=4, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")
    plt.xlabel("Training sample size")
    plt.ylabel("classification accuracy")
    plt.legend()

estimator = GaussianNB()
plot_learning_curve(estimator,"Naive Bayes",X,y,ylim=[0.75,1.05],cv=cv)

estimator = LinearSVC()
plot_learning_curve(estimator,"Support Vector Classification",X,y,ylim=[0.75,1.05],cv=cv)


## Reduce the number of features in neuropsych measures: factor analysis with R

Below, we're looping over a number of components and compute for each n the crossvalidation loglikelihood.

In [ ]:
# %%R
# leave this in: only way to install package with R --> will need :)
# install.packages("GPArotation",repos='http://cran.us.r-project.org')

In [ ]:
FA_table.shape

In the cell below, we're running a factor analysis in R.  The command in R gives us way more metrics than what we get with scikit-learn.  The output is `factortable`: a table with different metrics against a different number of factors.

**Note** This code took too long, and was deployed to a supercomputer.

In [ ]:
# %%R -i FA_table -o factortable
#
#maxfac <- 50
#
#library(psych)
#factortable <- nfactors(FA_table,n=maxfac,rotation='varimax')$vss.stats
#factortable['eigen'] <- eigen(cor(FA_table))$values[1:maxfac]
#
#parallel <- fa.parallel(FA_table,plot=FALSE)
#factortable['parallel_data'] <- parallel$fa.values[1:maxfac]
#factortable['parallel_sim'] <- parallel$fa.sim[1:maxfac]
#
#vss <- vss(FA_table,n=maxfac,rotate='varimax',plot=FALSE)
#factortable['map'] <- vss$map

factortable = pd.read_csv("04_connectome/FA_table_outR.csv",sep=" ")

In [ ]:
factortable[:5]

In [ ]:
# chi square of residual matrix
pvals = [stats.chi2.pdf(x,y) for x,y in zip(factortable.chisq,factortable.dof)]
num_c = sum(np.array(pvals)<0.05)
print("Based on the likelihood ration test, Model(N)-Model(0), the minimum number of factors is %i"%num_c)

# likelihood ration
difs = np.diff(factortable.chisq)
difdofs = np.diff(factortable.dof)
pvals = [stats.chi2.pdf(-x,-y) for x,y in zip(difs,difdofs)]
num_c = sum(np.array(pvals)<0.05)
print("Based on the likelihood ration test, Model(b)-Model(n-1), the minimum number of factors is %i"%(num_c+1))

# RMSEA
num_r = sum(factortable.RMSEA>0.05)
print("Based on the RMSEA, the minimum number of factors is %i"%(num_r))


#kaiser criterion: number of eigenvalues > 1
num_k = sum(factortable['eigen']>1)
print("Based on the kaiser criterion, the minimum number of factors is %i"%(num_r))

# BIC
num_k = np.where(np.min(factortable.BIC) == factortable.BIC)[0][0]+1
print("Based on the BIC, the minimum number of factors is %i"%(num_k))
num_k = np.where(np.min(factortable.eBIC) == factortable.eBIC)[0][0]+1
print("Based on the empirical BIC, the minimum number of factors is %i"%(num_k))

# complexity
num_c = np.where(np.max(factortable.complex)==factortable.complex)[0][0]+1
print("Based on the complexity, the minimum number of factors is %i"%(num_c))

# SRMR (standardised root mean square residual)
num_c = sum(factortable.SRMR<0.08)
print("Based on the standardised root mean square residual, the minimum number of factors is %i"%(num_c))

# parallel analysis: extract factors until eigen values of data < eigen vanlues of random data
#num_c = np.min(np.where(factortable.parallel_data<factortable.parallel_sim)[0])+1
#print("Based on parallel analysis, the minimum number of factors is %i"%(num_c))

# map: minimum average partial correlation
num_c = np.where(np.min(factortable.map)==factortable.map)[0][0]
print("Based on the minimum average partial correlation method, the minimum number of factors is %i"%(num_c))

In [ ]:
sns.set_style("white")

factortable_norm = (factortable - factortable.mean()) / factortable.std()

fig = plt.figure(figsize=(10,10))
for idx,col in enumerate(factortable_norm):
    plt.plot(factortable_norm[col],label=col,color=cols[idx],lw=3)
plt.legend(loc='upper right')

### What does scikit-learn (based on cross-validation) say?

In [ ]:
%%R -i FA_table -o factortable

factors <- 5

#print(FA_table[1])
anal <- factanal(FA_table,factors=10,rotate='varimax',scores='Bartlett')

In [ ]:
fa.iterated_power=3
fa.n_components = 5
fa.fit(X)

In [ ]:
def promax(A,k=2):
    assert k>0
    V, T = rotate_factors(A,'varimax')
    H = np.abs(V)**k/V
    S=procrustes(A,H) #np.linalg.inv(A.T.dot(A)).dot(A.T).dot(H);
    d=np.sqrt(np.diag(np.linalg.inv(S.T.dot(S))));
    D=np.diag(d)
    T=np.linalg.inv(S.dot(D)).T
    return A.dot(T), T


In [ ]:
def varimax(Phi, gamma = 1, q = 20, tol = 1e-6):
    from numpy import eye, asarray, dot, sum, diag
    from numpy.linalg import svd
    p,k = Phi.shape
    R = eye(k)
    d=0
    for i in xrange(q):
        d_old = d
        Lambda = dot(Phi, R)
        u,s,vh = svd(dot(Phi.T,asarray(Lambda)**3 - (gamma/p) * dot(Lambda, diag(diag(dot(Lambda.T,Lambda))))))
        R = dot(u,vh)
        d = sum(s)
        if d/d_old < tol: break
    return dot(Phi, R)

In [ ]:
X_red = varimax(fa.components_)
#X_red = fa.components_

In [ ]:
minor_ticks = np.arange(len(labeltable.code))

labelnew = [x[:50] for x in labeltable.label]
fig = plt.figure(figsize=(10, 30), dpi= 100, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111)
ax1 = ax.imshow(np.transpose(X_red),
                aspect='auto',interpolation='nearest',cmap='coolwarm')
ax.set_yticks(np.arange(X_red.shape[1]),minor=True)
ax.set_yticklabels(labelnew,minor=True)
plt.colorbar(ax1)


coldict = {k:cols[idx] for idx,k in enumerate(np.unique(labeltable.scale))}
labeltable['colors'] = [coldict[x] for x in labeltable.scale]
for ytick, color in zip(ax.get_yticklabels(minor=True), labeltable.colors):
    ytick.set_color(color)

plt.tight_layout()

In [ ]:
np.arange(len(neuropsych_cols))[:100]

In [ ]:
upid = np.triu_indices(statT.shape[0])
len(upid[0])

In [ ]:
connectomes = np.load(os.path.join(CONDIR,'derivatives/connectomes.npy'))
passid = np.where(np.logical_and(table.MOTION_pass == 1, table.MRIQC_pass == 1))[0]
connectomes = connectomes[:,:,passid]
subtable = table.iloc[list(passid),:]
subtable = subtable.reset_index()

In [ ]:
wide = np.zeros([len(passid),len(upid[0])])
for sub in range(len(passid)):
    wide[sub,:] = connectomes[:,:,sub][upid]
wide = pd.DataFrame(wide)
print(wide.shape)

### Canonical correlation

In [ ]:
from sklearn.cross_decomposition import CCA
cca = CCA(n_components = 4)
cca.fit(X_new,Y)

In [ ]:
cca.get_params()
#cca.x_scores_.shape
cca.x_weights_.shape

In [ ]:
cca.y_weights_

In [ ]:
def rotate_factors(A, method, *method_args, **algorithm_kwargs):
    r"""
    Subroutine for orthogonal and oblique rotation of the matrix :math:`A`.
    For orthogonal rotations :math:`A` is rotated to :math:`L` according to
    
    .. math::
        L =  AT,
        
    where :math:`T` is an orthogonal matrix. And, for oblique rotations
    :math:`A` is rotated to :math:`L` according to
    
    .. math::
        L =  A(T^*)^{-1},
    where :math:`T` is a normal matrix.
    
    Methods
    -------
    What follows is a list of available methods. Depending on the method
    additional argument are required and different algorithms
    are available. The algorithm_kwargs are additional keyword arguments
    passed to the selected algorithm (see the parameters section).
    Unless stated otherwise, only the gpa and
    gpa_der_free algorithm are available.
    
    Below,
    
    * :math:`L` is a :math:`p\times k` matrix;
    * :math:`N` is :math:`k\times k` matrix with zeros on the diagonal and ones elsewhere;
    * :math:`M` is :math:`p\times p` matrix with zeros on the diagonal and ones elsewhere;
    * :math:`C` is a :math:`p\times p` matrix with elements equal to :math:`1/p`;
    * :math:`(X,Y)=\operatorname{Tr}(X^*Y)` is the Frobenius norm;
    * :math:`\circ` is the element-wise product or Hadamard product.
    
    oblimin : orthogonal or oblique rotation that minimizes
        .. math::
            \phi(L) = \frac{1}{4}(L\circ L,(I-\gamma C)(L\circ L)N).
            
        For orthogonal rotations:
        * :math:`\gamma=0` corresponds to quartimax,
        * :math:`\gamma=\frac{1}{2}` corresponds to biquartimax,
        * :math:`\gamma=1` corresponds to varimax,
        * :math:`\gamma=\frac{1}{p}` corresponds to equamax.
        For oblique rotations rotations:
    
        * :math:`\gamma=0` corresponds to quartimin,
        * :math:`\gamma=\frac{1}{2}` corresponds to biquartimin.
        
        method_args:
        
        gamma : float
            oblimin family parameter
        rotation_method : string
            should be one of {orthogonal, oblique}
    
    orthomax : orthogonal rotation that minimizes
        .. math::
            \phi(L) = -\frac{1}{4}(L\circ L,(I-\gamma C)(L\circ L)),
            
        where :math:`0\leq\gamma\leq1`. The orthomax family is equivalent to 
        the oblimin family (when restricted to orthogonal rotations). Furthermore,
        * :math:`\gamma=0` corresponds to quartimax,
        * :math:`\gamma=\frac{1}{2}` corresponds to biquartimax,
        * :math:`\gamma=1` corresponds to varimax,
        * :math:`\gamma=\frac{1}{p}` corresponds to equamax.
        
        method_args:
        
        gamma : float (between 0 and 1)
            orthomax family parameter
    
    CF : Crawford-Ferguson family for orthogonal and oblique rotation wich minimizes:
        .. math::
            \phi(L) =\frac{1-\kappa}{4} (L\circ L,(L\circ L)N)
                      -\frac{1}{4}(L\circ L,M(L\circ L)),
                      
        where :math:`0\leq\kappa\leq1`. For orthogonal rotations the oblimin
        (and orthomax) family of rotations is equivalent to the Crawford-Ferguson family.
        To be more precise:
    
        * :math:`\kappa=0` corresponds to quartimax,
        * :math:`\kappa=\frac{1}{p}` corresponds to varimax,
        * :math:`\kappa=\frac{k-1}{p+k-2}` corresponds to parsimax,
        * :math:`\kappa=1` corresponds to factor parsimony.
        
        method_args:
        
        kappa : float (between 0 and 1)
            Crawford-Ferguson family parameter
        rotation_method : string
            should be one of {orthogonal, oblique}
    
    quartimax : orthogonal rotation method
        minimizes the orthomax objective with :math:`\gamma=0`
        
    biquartimax : orthogonal rotation method
        minimizes the orthomax objective with :math:`\gamma=\frac{1}{2}`
        
    varimax : orthogonal rotation method
        minimizes the orthomax objective with :math:`\gamma=1`
        
    equamax : orthogonal rotation method
        minimizes the orthomax objective with :math:`\gamma=\frac{1}{p}`
        
    parsimax : orthogonal rotation method
        minimizes the Crawford-Ferguson family objective with :math:`\kappa=\frac{k-1}{p+k-2}`
        
    parsimony : orthogonal rotation method
        minimizes the Crawford-Ferguson family objective with :math:`\kappa=1`
    
    quartimin : oblique rotation method that minimizes
        minimizes the oblimin objective with :math:`\gamma=0`      
    quartimin : oblique rotation method that minimizes
        minimizes the oblimin objective with :math:`\gamma=\frac{1}{2}`   
        
    target : orthogonal or oblique rotation that rotates towards a target matrix :math:`H` by minimizing the objective
        .. math::
            \phi(L) =\frac{1}{2}\|L-H\|^2.
        
        method_args:
        
        H : numpy matrix
            target matrix
        rotation_method : string
            should be one of {orthogonal, oblique}
        For orthogonal rotations the algorithm can be set to analytic in which case
        the following keyword arguments are available:
        
        full_rank : boolean (default False)
            if set to true full rank is assumed    
    partial_target : orthogonal (default) or oblique rotation that partially rotates
        towards a target matrix :math:`H` by minimizing the objective:
        
        .. math::
            \phi(L) =\frac{1}{2}\|W\circ(L-H)\|^2.
        
        method_args:
        
        H : numpy matrix
            target matrix
        W : numpy matrix (default matrix with equal weight one for all entries)
            matrix with weights, entries can either be one or zero
    
    Parameters
    ---------
    A : numpy matrix (default None)
        non rotated factors
    method : string
        should be one of the methods listed above
    method_args : list
        additional arguments that should be provided with each method
    algorithm_kwargs : dictionary
        algorithm : string (default gpa)
            should be one of:
            
            * 'gpa': a numerical method
            * 'gpa_der_free': a derivative free numerical method
            * 'analytic' : an analytic method
        Depending on the algorithm, there are algorithm specific keyword
        arguments. For the gpa and gpa_der_free, the following
        keyword arguments are available:
        
        max_tries : integer (default 501)
            maximum number of iterations
        tol : float
            stop criterion, algorithm stops if Frobenius norm of gradient is
            smaller then tol
        For analytic, the supporeted arguments depend on the method, see above.
            
        See the lower level functions for more details.
        
    Returns
    -------
    The tuple :math:`(L,T)`
    
    Examples
    -------
    >>> A = np.random.randn(8,2)
    >>> L, T = rotate_factors(A,'varimax')
    >>> np.allclose(L,A.dot(T))
    >>> L, T = rotate_factors(A,'orthomax',0.5)
    >>> np.allclose(L,A.dot(T))
    >>> L, T = rotate_factors(A,'quartimin',0.5)
    >>> np.allclose(L,A.dot(np.linalg.inv(T.T)))
    """
    if 'algorithm' in algorithm_kwargs:
        algorithm = algorithm_kwargs['algorithm']
        algorithm_kwargs.pop('algorithm')
    else:
        algorithm = 'gpa'
    assert 'rotation_method' not in algorithm_kwargs, 'rotation_method cannot be provided as keyword argument'
    L=None
    T=None
    ff=None
    vgQ=None
    p,k = A.shape
    #set ff or vgQ to appropriate objective function, compute solution using recursion or analytically compute solution
    if method == 'orthomax':
        assert len(method_args)==1, 'Only %s family parameter should be provided' % method
        rotation_method='orthogonal'
        gamma = method_args[0]
        if algorithm =='gpa':
            vgQ=lambda L=None, A=None, T=None: gr.orthomax_objective(L=L,A=A,T=T,
                                                                     gamma=gamma,
                                                                     return_gradient=True)
        elif algorithm =='gpa_der_free':
            ff = lambda L=None, A=None, T=None: gr.orthomax_objective(L=L,A=A,T=T,
                                                                      gamma=gamma,
                                                                      return_gradient=False)
        else:
            raise ValueError('Algorithm %s is not possible for %s rotation' % (algorithm, method))
    elif method == 'oblimin':
        assert len(method_args)==2, 'Both %s family parameter and rotation_method should be provided' % method
        rotation_method=method_args[1]
        assert rotation_method in ['orthogonal','oblique'], 'rotation_method should be one of {orthogonal, oblique}'
        gamma = method_args[0]
        if algorithm =='gpa':
            vgQ=lambda L=None, A=None, T=None: gr.oblimin_objective(L=L,A=A,T=T,
                                                                    gamma=gamma,
                                                                    return_gradient=True)
        elif algorithm =='gpa_der_free':
            ff = lambda L=None, A=None, T=None: gr.oblimin_objective(L=L,A=A,T=T,
                                                                     gamma=gamma,
                                                                     rotation_method=rotation_method,
                                                                     return_gradient=False)
        else:
            raise ValueError('Algorithm %s is not possible for %s rotation' % (algorithm, method))
    elif method == 'CF':
        assert len(method_args)==2, 'Both %s family parameter and rotation_method should be provided' % method
        rotation_method=method_args[1]
        assert rotation_method in ['orthogonal','oblique'], 'rotation_method should be one of {orthogonal, oblique}'
        kappa = method_args[0]
        if algorithm =='gpa':
            vgQ=lambda L=None, A=None, T=None: gr.CF_objective(L=L,A=A,T=T,
                                                               kappa=kappa,
                                                               rotation_method=rotation_method,
                                                               return_gradient=True)
        elif algorithm =='gpa_der_free':
            ff = lambda L=None, A=None, T=None: gr.CF_objective(L=L,A=A,T=T,
                                                                kappa=kappa,
                                                                rotation_method=rotation_method,
                                                                return_gradient=False)
        else:
            raise ValueError('Algorithm %s is not possible for %s rotation' % (algorithm, method))
    elif method == 'quartimax':
        return rotate_factors(A, 'orthomax', 0, **algorithm_kwargs)
    elif method == 'biquartimax':
        return rotate_factors(A, 'orthomax', 0.5, **algorithm_kwargs)
    elif method == 'varimax':
        return rotate_factors(A, 'orthomax', 1, **algorithm_kwargs)
    elif method == 'equamax':
        return rotate_factors(A, 'orthomax', 1/p, **algorithm_kwargs)
    elif method == 'parsimax':
        return rotate_factors(A, 'CF', (k-1)/(p+k-2), 'orthogonal', **algorithm_kwargs)    
    elif method == 'parsimony':
        return rotate_factors(A, 'CF', 1, 'orthogonal', **algorithm_kwargs)    
    elif method == 'quartimin':
        return rotate_factors(A, 'oblimin', 0, 'oblique', **algorithm_kwargs)
    elif method == 'biquartimin':
        return rotate_factors(A, 'oblimin', 0.5, 'oblique', **algorithm_kwargs)
    elif method == 'target':
        assert len(method_args)==2, 'only the rotation target and orthogonal/oblique should be provide for %s rotation' % method
        H=method_args[0]
        rotation_method=method_args[1]
        assert rotation_method in ['orthogonal','oblique'], 'rotation_method should be one of {orthogonal, oblique}'
        if algorithm =='gpa':
            vgQ=lambda L=None, A=None, T=None: gr.vgQ_target(H,L=L,A=A,T=T,rotation_method=rotation_method)
        elif algorithm =='gpa_der_free':
            ff = lambda L=None, A=None, T=None: gr.ff_target(H,L=L,A=A,T=T,rotation_method=rotation_method)
        elif algorithm =='analytic':
            assert rotation_method == 'orthogonal', 'For analytic %s rotation only orthogonal rotation is supported'
            T= ar.target_rotation(A,H,**algorithm_kwargs)
        else:
            raise ValueError('Algorithm %s is not possible for %s rotation' % (algorithm, method))
    elif method == 'partial_target':
        assert len(method_args)==2, '2 additional arguments are expected for %s rotation' % method
        H=method_args[0]
        W=method_args[1]
        rotation_method='orthogonal'
        if algorithm =='gpa':
            vgQ=lambda L=None, A=None, T=None: gr.vgQ_partial_target(H,W=W,L=L,A=A,T=T)
        elif algorithm =='gpa_der_free':
            ff = lambda L=None, A=None, T=None: gr.ff_partial_target(H,W=W,L=L,A=A,T=T)
        else:
            raise ValueError('Algorithm %s is not possible for %s rotation' % (algorithm, method))
    else:
        raise ValueError('Invalid method')
    #compute L and T if not already done
    if T is None:
        L, phi, T, table = gr.GPA(A, vgQ=vgQ, ff=ff, rotation_method=rotation_method, **algorithm_kwargs)
    if L is None:
        assert T is not None, 'Cannot compute L without T'
        L=gr.rotateA(A,T,rotation_method=rotation_method)
    #return
    return L, T
